## Vektorizace

V našem předchozím příkladu jsme pracovali s vysoko-dimenzionálními vektory bag-of-words o délce `vocab_size` a explicitně jsme převáděli nízko-dimenzionální poziční reprezentace na řídké one-hot reprezentace. Tato one-hot reprezentace není paměťově efektivní, navíc je každý slovní prvek zpracováván nezávisle na ostatních, tj. one-hot kódované vektory nevyjadřují žádnou sémantickou podobnost mezi slovy.

V této části budeme pokračovat v průzkumu datasetu **News AG**. Nejprve načtěme data a získáme některé definice z předchozího notebooku.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Co je to embedding?

Myšlenka **embeddingu** spočívá v reprezentaci slov pomocí nižší dimenze hustých vektorů, které nějakým způsobem odrážejí sémantický význam slova. Později si povíme, jak vytvořit smysluplné word embeddings, ale prozatím si představme embeddingy jako způsob, jak snížit dimenzionalitu vektoru slova.

Embeddingová vrstva tedy přijme slovo jako vstup a vytvoří výstupní vektor o specifikované velikosti `embedding_size`. V jistém smyslu je velmi podobná vrstvě `Linear`, ale místo toho, aby přijímala vektor zakódovaný metodou one-hot, dokáže přijmout číslo slova jako vstup.

Použitím embeddingové vrstvy jako první vrstvy v naší síti můžeme přejít od modelu bag-of-words k modelu **embedding bag**, kde nejprve převedeme každé slovo v našem textu na odpovídající embedding a poté vypočítáme nějakou agregační funkci přes všechny tyto embeddingy, například `sum`, `average` nebo `max`.

![Obrázek ukazující klasifikátor embeddingu pro pět slov v sekvenci.](../../../../../translated_images/cs/embedding-classifier-example.b77f021a7ee67eee.webp)

Naše klasifikační neuronová síť začne embeddingovou vrstvou, poté agregační vrstvou a na vrcholu bude lineární klasifikátor:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Práce s proměnnou délkou sekvence

V důsledku této architektury je nutné vytvářet minibatch pro naši síť určitým způsobem. V předchozí části, při použití metody bag-of-words, měly všechny BoW tenzory v minibatch stejnou velikost `vocab_size`, bez ohledu na skutečnou délku textové sekvence. Jakmile přejdeme na slovní vektory (word embeddings), skončíme s proměnným počtem slov v každém textovém vzorku, a při kombinování těchto vzorků do minibatch bude nutné použít nějaké doplnění (padding).

To lze provést pomocí stejné techniky, kdy se funkce `collate_fn` poskytne datovému zdroji:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Trénování klasifikátoru embeddingů

Nyní, když jsme definovali správný dataloader, můžeme model natrénovat pomocí trénovací funkce, kterou jsme definovali v předchozí části:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Poznámka**: Zde trénujeme pouze na 25 tisíc záznamů (méně než jeden celý epoch) kvůli úspoře času, ale můžete pokračovat v tréninku, napsat funkci pro trénink na několik epoch a experimentovat s parametrem rychlosti učení, abyste dosáhli vyšší přesnosti. Měli byste být schopni dosáhnout přesnosti kolem 90 %.


### Vrstva EmbeddingBag a reprezentace sekvencí s proměnnou délkou

V předchozí architektuře jsme museli všechny sekvence doplnit na stejnou délku, aby se vešly do minibatch. To není nejefektivnější způsob, jak reprezentovat sekvence s proměnnou délkou – jiný přístup by byl použití **offsetového** vektoru, který by obsahoval offsety všech sekvencí uložených v jednom velkém vektoru.

![Obrázek znázorňující reprezentaci sekvencí pomocí offsetů](../../../../../translated_images/cs/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: Na obrázku výše je znázorněna sekvence znaků, ale v našem příkladu pracujeme se sekvencemi slov. Nicméně obecný princip reprezentace sekvencí pomocí offsetového vektoru zůstává stejný.

Pro práci s offsetovou reprezentací používáme vrstvu [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). Je podobná vrstvě `Embedding`, ale jako vstup bere obsahový vektor a offsetový vektor, a navíc zahrnuje vrstvu pro průměrování, která může být `mean`, `sum` nebo `max`.

Zde je upravená síť, která používá `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Pro přípravu datové sady pro trénink musíme poskytnout konverzní funkci, která připraví vektor offsetu:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Všimněte si, že na rozdíl od všech předchozích příkladů naše síť nyní přijímá dva parametry: datový vektor a vektor offsetu, které mají různé velikosti. Podobně nám náš datový nakladač poskytuje 3 hodnoty místo 2: jako vlastnosti jsou poskytovány jak textové, tak offsetové vektory. Proto musíme mírně upravit naši trénovací funkci, aby to zohlednila:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Sémantické vektory: Word2Vec

V našem předchozím příkladu se vrstva modelu pro vektorizaci naučila mapovat slova na jejich vektorovou reprezentaci, avšak tato reprezentace neměla příliš velký sémantický význam. Bylo by skvělé naučit se takovou vektorovou reprezentaci, kde by podobná slova nebo synonyma odpovídala vektorům, které jsou blízko sebe podle určité vektorové vzdálenosti (např. euklidovské vzdálenosti).

Abychom toho dosáhli, musíme náš model pro vektorizaci předem natrénovat na velké kolekci textů specifickým způsobem. Jedním z prvních způsobů, jak trénovat sémantické vektory, je metoda nazvaná [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Ta je založena na dvou hlavních architekturách, které se používají k vytvoření distribuované reprezentace slov:

 - **Continuous bag-of-words** (CBoW) — v této architektuře trénujeme model, aby předpovídal slovo na základě okolního kontextu. Při daném ngramu $(W_{-2},W_{-1},W_0,W_1,W_2)$ je cílem modelu předpovědět $W_0$ na základě $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** je opakem CBoW. Model využívá okolní okno kontextových slov k předpovědi aktuálního slova.

CBoW je rychlejší, zatímco skip-gram je pomalejší, ale lépe reprezentuje méně častá slova.

![Obrázek ukazující algoritmy CBoW a Skip-Gram pro převod slov na vektory.](../../../../../translated_images/cs/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Pro experimentování s Word2Vec vektory předem natrénovanými na datasetu Google News můžeme použít knihovnu **gensim**. Níže najdeme slova nejpodobnější slovu 'neural'.

> **Note:** Když poprvé vytváříte vektorové reprezentace slov, jejich stahování může chvíli trvat!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Můžeme také vypočítat vektorové embeddingy ze slova, které budou použity při trénování klasifikačního modelu (pro přehlednost zobrazujeme pouze prvních 20 komponent vektoru):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Skvělá věc na sémantických vnořeních je, že můžete manipulovat s vektorovým kódováním, abyste změnili sémantiku. Například můžeme požádat o nalezení slova, jehož vektorová reprezentace by byla co nejblíže slovům *král* a *žena* a co nejdále od slova *muž*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Oba modely, CBoW i Skip-Grams, jsou „prediktivní“ vektorizace, protože berou v úvahu pouze lokální kontexty. Word2Vec nevyužívá globální kontext.

**FastText** staví na Word2Vec tím, že se učí vektorové reprezentace pro každé slovo a n-gramy znaků, které se v daném slově nacházejí. Hodnoty těchto reprezentací se pak při každém kroku trénování zprůměrují do jednoho vektoru. I když to přidává značné množství výpočetní náročnosti při předtrénování, umožňuje to vektorizacím slov zakódovat informace o podslovech.

Další metoda, **GloVe**, využívá myšlenku matice společného výskytu a používá neuronové metody k dekompozici této matice do expresivnějších a nelineárních vektorů slov.

Můžete si vyzkoušet příklad tím, že změníte vektorizace na FastText a GloVe, protože gensim podporuje několik různých modelů vektorizace slov.


## Použití předtrénovaných vektorů v PyTorch

Můžeme upravit výše uvedený příklad tak, aby byla matice v naší vrstvě embedding předem naplněna sémantickými vektory, jako je Word2Vec. Musíme vzít v úvahu, že slovníky předtrénovaných vektorů a našeho textového korpusu se pravděpodobně nebudou shodovat, takže inicializujeme váhy pro chybějící slova náhodnými hodnotami:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Nyní pojďme natrénovat náš model. Všimněte si, že doba potřebná k natrénování modelu je výrazně delší než v předchozím příkladu, a to kvůli větší velikosti vrstvy vnoření, a tím pádem mnohem vyššímu počtu parametrů. Také z tohoto důvodu možná budeme muset natrénovat náš model na více příkladech, pokud se chceme vyhnout přeučení.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

V našem případě nevidíme výrazné zvýšení přesnosti, což je pravděpodobně způsobeno velmi odlišnými slovníky.  
Abychom překonali problém odlišných slovníků, můžeme použít jedno z následujících řešení:  
* Znovu natrénovat model word2vec na našem slovníku  
* Načíst náš dataset se slovníkem z předtrénovaného modelu word2vec. Slovník použitý k načtení datasetu lze specifikovat během načítání.  

Druhý přístup se zdá být jednodušší, zejména proto, že framework `torchtext` v PyTorch obsahuje vestavěnou podporu pro embeddingy. Můžeme například vytvořit slovník založený na GloVe následujícím způsobem:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Načtená slovní zásoba má následující základní operace:
* Slovník `vocab.stoi` nám umožňuje převést slovo na jeho index ve slovníku
* `vocab.itos` dělá opak - převádí číslo na slovo
* `vocab.vectors` je pole vektorů pro vkládání, takže pro získání vektoru pro slovo `s` musíme použít `vocab.vectors[vocab.stoi[s]]`

Zde je příklad manipulace s vektory pro demonstraci rovnice **kind-man+woman = queen** (musel jsem trochu upravit koeficient, aby to fungovalo):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

K trénování klasifikátoru pomocí těchto vektorových reprezentací musíme nejprve zakódovat náš dataset pomocí slovníku GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Jak jsme viděli výše, všechny vektorové embeddingy jsou uloženy v matici `vocab.vectors`. To umožňuje velmi snadné načtení těchto vah do vah vrstvy embeddingu pomocí jednoduchého kopírování:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Nyní pojďme natrénovat náš model a zjistit, zda dosáhneme lepších výsledků:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Jedním z důvodů, proč nevidíme významné zvýšení přesnosti, je skutečnost, že některá slova z našeho datového souboru chybí v předtrénovaném slovníku GloVe, a proto jsou v podstatě ignorována. Abychom tuto skutečnost překonali, můžeme na našem datovém souboru natrénovat vlastní vektory slov.


## Kontextuální vektory slov

Jedním z hlavních omezení tradičních předtrénovaných reprezentací vektorů slov, jako je Word2Vec, je problém s rozlišením významu slov. Zatímco předtrénované vektory dokážou zachytit část významu slov v kontextu, každý možný význam slova je zakódován do stejného vektoru. To může způsobovat problémy v následných modelech, protože mnoho slov, například slovo „play“, má různé významy v závislosti na kontextu, ve kterém jsou použity.

Například slovo „play“ má v těchto dvou větách zcela odlišný význam:
- Šel jsem na **hru** do divadla.
- John si chce **hrát** se svými přáteli.

Předtrénované vektory výše reprezentují oba tyto významy slova „play“ stejným vektorem. Abychom toto omezení překonali, musíme vytvářet vektory založené na **jazykovém modelu**, který je natrénován na rozsáhlém korpusu textu a *rozumí*, jak lze slova skládat v různých kontextech. Diskuze o kontextuálních vektorech slov přesahuje rámec tohoto tutoriálu, ale vrátíme se k nim při probírání jazykových modelů v další části.



---

**Upozornění**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za závazný zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Neodpovídáme za jakékoli nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
